In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import string
import re
from tqdm import tqdm

import math
from collections import defaultdict


In [2]:
df_rec=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/MealRec/recipe.csv")

In [3]:
df_meal=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\MealRec\meal.csv")

In [4]:
appetizers=df_meal['appetizer'].values
main=df_meal['main_dish'].values
desert=df_meal['dessert'].values

In [5]:
ids=df_rec['recipe_id'].values
titles=df_rec['recipe_name'].values
id2title={ids[i]:titles[i] for i in range(len(ids))}
title2id={titles[i]:ids[i] for i in range(len(ids))}
id2id={ids[i]:i+1 for i in range(len(ids))}

In [17]:
sents=[]
for i in range(len(df_meal)):
    sents.append([
        id2id[appetizers[i]],
        id2id[main[i]],
        id2id[desert[i]],
    ])

In [18]:
len(sents)

3817

In [7]:
df_kgl=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Kaggle Meal Plan\mealplans.csv")

In [8]:
df_kgl_dishes=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Kaggle Meal Plan\dishes.csv")

In [9]:
remove_punc=['!','"','.','?',':','/','*','~','#','$','%',';','+','=','[',']','<','>','@','`','^','\\','_','{','}','|']
remove_punc_str="".join(remove_punc) 

def clean(s):
  s = s.title() # convert to titlecase: All individual words first letter capitalised
  s=s.strip() # Remove trailing and leading spaces
  s=s.lstrip(string.digits+" "+string.punctuation) # remove leading numbers, spaces and punctuations
  s = s.translate(str.maketrans({key: None for key in remove_punc_str})) # remove other unnecessary punctuations
  if(len(s)>3):
    if(s[:2]=="A " or s[:3]=="A'S" or s[:2]=="A-" or s[:2]=="A)" or s[:2]=="A1" or s[:3]=="(A)"):
      s=s[2:]
  else:
    return ''
  s=s.lstrip(string.digits+" "+string.punctuation) # remove leading numbers, spaces and punctuations
  s=s.rstrip(" "+remove_punc_str)  # remove trailing spaces and punctuations. Not digits as titles can end with <recipe> for 1
  s=re.sub(' +', ' ', s) # remove extra spaces from in between
  return s

In [11]:
kgl_titles=df_kgl_dishes['title'].values
kgl_ids=df_kgl_dishes['id'].values
kgl_titles=[clean(val) for val in kgl_titles]
kgl_id2title={kgl_ids[i]:kgl_titles[i] for i in range(len(kgl_titles))}

In [12]:
meal_plans=df_kgl[['breakfast0', 'breakfast1', 'breakfast2', 'lunch0', 'lunch1', 'lunch2',
       'lunch3', 'lunch4', 'dinner0', 'dinner1', 'dinner2', 'dinner3',
       'dinner4', 'dinner5', 'dinner6', 'snacks0', 'snacks1']].values.tolist()

In [19]:
df_recs=pd.read_csv("F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\dishes.csv")

In [20]:
titles=df_recs['Title'].values
titles=[clean(val) for val in titles]
title2id={titles[i]:i+1 for i in range(len(titles))}
titles=set(titles)

In [21]:
sentences=[]
for i in range(len(df_meal)):
    sentences.append([
        id2id[appetizers[i]],
        id2id[main[i]],
        id2id[desert[i]],
    ])

In [22]:
for meal_plan in tqdm(meal_plans):
    flag=1
    sent=[]
    for dish in meal_plan:
        if(not np.isnan(float(dish))):
            if kgl_id2title[int(dish)] in titles:
                sent.append(title2id[kgl_id2title[int(dish)]])
    if len(sent)>1:
        sentences.append(sent)

100%|██████████| 275290/275290 [00:12<00:00, 21602.42it/s]


In [23]:
len(sentences)

110301

In [33]:
df_recs.head()

,ID,Title,Category,Cuisine,Calories,Fats,Proteins,Carbohydrates,Breakfast,Lunch,Snacks,Dinner,Typical_serving_size
0,1,Mom'S Zucchini Bread,appetizer,North America,255.1692,13.140870,3.268513,32.05494,1,0,0,0,100
1,2,Zucchini Walnut Bread,appetizer,NaN,276.0908,13.398670,3.977222,36.54568,1,0,0,0,100
2,3,Honey Wheat Bread I,appetizer,NaN,156.4551,4.770610,4.626510,24.45452,0,1,0,1,100
3,4,Chocolate Chip Orange Zucchini Bread,dessert,NaN,280.3475,15.283680,3.563895,34.25740,1,0,0,0,100
4,5,Chocolate Chip Pumpkin Bread,dessert,NaN,210.1581,9.410045,2.556006,30.47000,1,0,0,0,100


In [34]:
titles=df_recs['Title'].values
breakfast=df_recs['Breakfast'].values
lunch=df_recs['Lunch'].values
snacks=df_recs['Snacks'].values
dinner=df_recs['Dinner'].values
titles=[clean(val) for val in titles]
id2title={i+1:titles[i] for i in range(len(titles))}

In [28]:
def normalize(d):
    factor=max(d.items(),key=lambda x:x[1])[1]
    for k in d:
        d[k]/=factor   

In [35]:
scores_breakfast=defaultdict(float)
scores_lunch=defaultdict(float)
scores_snacks=defaultdict(float)
scores_dinner=defaultdict(float)

for sent in sentences:
    for i in range(len(sent)):
        for j in range(len(sent)):
            if(i!=j):
                i1=sent[i]-1
                i2=sent[j]-1
                if(breakfast[i1]==1 and breakfast[i2]==1):
                    scores_breakfast[(i1+1,i2+1)]+=1
                
                if(lunch[i1]==1 and lunch[i2]==1):
                    scores_lunch[(i1+1,i2+1)]+=1

                if(snacks[i1]==1 and snacks[i2]==1):
                    scores_snacks[(i1+1,i2+1)]+=1

                if(dinner[i1]==1 and dinner[i2]==1):
                    scores_dinner[(i1+1,i2+1)]+=1

In [37]:
breakfast_items=[i for i in range(len(df_recs)) if breakfast[i]==1]
lunch_items=[i for i in range(len(df_recs)) if lunch[i]==1]
snacks_items=[i for i in range(len(df_recs)) if snacks[i]==1]
dinner_items=[i for i in range(len(df_recs)) if dinner[i]==1]

In [39]:
len(breakfast_items),len(lunch_items),len(snacks_items),len(dinner_items)

(192, 4604, 829, 5866)

In [155]:
def makeGraph(items,scores,file,a):
    G=nx.Graph()

    G.add_nodes_from(items)

    normalize(scores)
    avg=math.fsum(scores.values())/len(scores)

    edges=[]
    for sc in scores.items():
        if sc[1]>avg*a:
            edges.append((sc[0][0],sc[0][1],sc[1]))

    G.add_weighted_edges_from(edges)

    print(len(G.edges))

    nx.write_edgelist(G,"F:\SHRINIVAS\KGP\BTP\Meal-Planning\Data\Processed\graphs\\"+file+".edgelist")

In [167]:
makeGraph(breakfast_items,scores_breakfast,"breakfast",1)

92


In [158]:
makeGraph(lunch_items,scores_lunch,"lunch",3)

2082


In [159]:
makeGraph(snacks_items,scores_snacks,"snacks",2)

201


In [160]:
makeGraph(dinner_items,scores_dinner,"dinner",3)

2552


In [104]:
G=nx.read_edgelist("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/graphs/breakfast.edgelist")

In [105]:
cliques=list(nx.enumerate_all_cliques(G))

In [106]:
clean=[]
for c in cliques:
    if(len(c)>1 and len(c)<=3):
        clean.append(c)

In [107]:
len(clean)

952

In [101]:
clean[0]

['1', '2547']

In [83]:
titles[0],titles[2546],titles[2148]

("Mom'S Zucchini Bread", 'Banana Banana Bread', 'Chocolate Zucchini Bread I')

In [124]:
def getClique(file,a,b):
    G=nx.read_edgelist("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/graphs//"+file+".edgelist")
    
    cliques=list(nx.enumerate_all_cliques(G))

    clean=[]
    for c in cliques:
        if(len(c)>a and len(c)<=b):
            clean.append(c)

    return clean

In [161]:
len(getClique("lunch",3,5))

1229

In [168]:
len(getClique("breakfast",1,3))

129

In [150]:
def getTitles(clique):
    for id in clique:
        print(titles[int(id)-1])

In [ ]:
df_ings=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/ingredients.csv")

In [1]:
import pandas as pd

In [4]:
df_dishes=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/dishes.csv")
df_ings=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/ingredients.csv")
df_dish_ings=pd.read_csv("F:/SHRINIVAS/KGP/BTP/Meal-Planning/Data/Processed/rec_ing.csv")

In [6]:
titles=df_dishes['Title'].values
ingreds=df_ings['Name'].values

dishes=df_dishes.to_dict('records')
id2dish={}
for dish in dishes:
    id2dish[dish['ID']]={
        'Title': dish['Title'],
        'Nutrition': [dish['Calories'],dish['Fats'],dish['Proteins'],dish['Carbohydrates']],
        'Serving Size': dish['Typical_serving_size'],
        'Ingredients': ''
    }

dish=df_dish_ings['Recipe ID'].values
ings=df_dish_ings['Ingredient ID'].values
for i in range(len(dish)):
    id2dish[dish[i]]['Ingredients']+=ingreds[ings[i]-1]+"^"

In [7]:
id2dish[2957]

{'Title': 'Tortilla Casserole',
 'Nutrition': [277.0868, 16.68324, 15.51585, 17.62903],
 'Serving Size': 100,
 'Ingredients': 'corn tortilla chips^monosodium glutamate (msg)^cheap beer^eggplants^mild taco^salmon steaks^'}